In [383]:
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn import preprocessing

In [384]:
test = pd.read_csv('Data/test.csv')
train = pd.read_csv('Data/train.csv')

In [385]:
mean_Age = train['Age'].mean()

In [386]:
#sns.lmplot('SibSp','Survived',train)

In [387]:
train.loc[train["SibSp"] == 8,"SibSp"] = 5
test.loc[test["SibSp"] == 8,"SibSp"] = 5

In [388]:
train.loc[train["Cabin"] != 0,"Cabin"] = 1
test.loc[test["Cabin"] != 0,"Cabin"] = 1

In [389]:
train.loc[train["Sex"] == "male","Sex"] = 0
train.loc[train["Sex"] == "female","Sex"] = 1
test.loc[test["Sex"] == "male","Sex"] = 0
test.loc[test["Sex"] == "female","Sex"] = 1

In [390]:
test = test.fillna(0)
train = train.fillna(0)

In [391]:
train.loc[train["Age"] == 0,"Age"] = mean_Age
test.loc[test["Age"] == 0,"Age"] = mean_Age

In [392]:
Surv_train = train['Survived']
Id = test['PassengerId']
train.drop(['Survived','Name','Ticket','Embarked','PassengerId'], axis='columns', inplace=True)
test.drop(['Name','Ticket','Embarked','PassengerId'], axis='columns', inplace=True)

In [393]:
normalized_train = preprocessing.normalize(train)
normalized_test = preprocessing.normalize(test)

In [394]:
decision = ExtraTreesClassifier(n_estimators=800)

In [395]:
#X_train, X_test, y_train, y_test = train_test_split(normalized_train, Surv_train, test_size=0.25, random_state=42)

In [396]:
#decision.fit(X_train, y_train)
#pred = decision.predict(X_test)

In [397]:
decision.fit(normalized_train, Surv_train)
pred = decision.predict(normalized_test)

In [398]:
#print('Accuracy: \n', accuracy_score(y_test, pred, normalize = True))

In [399]:
preds = pd.Series(pred)

In [400]:
d = {"PassengerId":Id, "Survived": preds}
df1 = pd.DataFrame(d)
df1.to_csv('answer.csv', index = False)

In [401]:
test = pd.read_csv('answer.csv')